<a href="https://colab.research.google.com/github/greapis/jaeha-shim/blob/main/Building_vision_model_by_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **학습할 내용**


이 Codelab에서 알아본 내용은 다음과 같습니다.

의류 기사를 인식하도록 신경망 학습시키기
네트워크의 다양한 레이어를 실험하는 과정을 안내하는 일련의 연습을 완료합니다.

## **빌드할 항목**

의류 물품을 식별하는 신경망